In [1]:
from utz import *
from nj_crashes.paths import ROOT_DIR
from njsp import crashes
from njsp.paths import PROJECTED_CSV
from njsp.ytc import to_ytc
from njsp.ytd import oldest_commit_rundate_since, projected_roy_deaths
from njsp import Ytd

chdir(ROOT_DIR)

In [2]:
ytd = Ytd()
prv_rundate = ytd.prv_rundate
prv_rundate

Searching for oldest commit with rundate ≥2024-11-17


Found rundate 2024-11-16 10:00:02 < 2024-11-17 at commit e7c82923; returning commit 4ef5110


'2024-11-17 10:00:04'

In [3]:
prv_ytd_ratio = ytd.cur_ytd_deaths / ytd.prv_ytd_deaths
pct_change = (prv_ytd_ratio - 1) * 100

print(f'As of {ytd.prv_rundate}, NJSP was reporting {ytd.prv_ytd_total} YTD deaths')
print(f'Current YTD Deaths ({ytd.rundate}): {ytd.cur_ytd_deaths}')
print(f'Previous year YTD Deaths (adjusted; {ytd.prv_rundate}): {ytd.prv_ytd_deaths}')
print(f'Projected {ytd.cur_year} total: {ytd.projected_year_total}')
print(f'{pct_change:.1f}% change')

As of 2024-11-17 10:00:04, NJSP was reporting 593 YTD deaths
Current YTD Deaths (2025-11-17 10:00:01-05:00): 504
Previous year YTD Deaths (adjusted; 2024-11-17 10:00:04): 592.7745816662494
Projected 2025 total: 583.2321603809349
-15.0% change


38731013fd583e06504bf1c9fb80557c703d632c: FAUQStats cache miss: 2024, Sun Nov 17 10:00:04 EST 2024


Find the oldest commit with rundate less than 1 year ago, load crashes from that time:

In [4]:
prv_crashes = ytd.prv_ytd_crashes
prv_crashes

,CCODE,CNAME,MCODE,MNAME,HIGHWAY,LOCATION,FATALITIES,FATAL_D,FATAL_P,FATAL_T,FATAL_B,STREET,INJURIES,dt
ACCID,,,,,,,,,,,,,,
12918,12,Middlesex,1218,Plainsboro Twsp,1,State Highway 1,1.0,0.0,0.0,1.0,0.0,NaN,4.0,2024-01-01 01:56:00-05:00
12923,11,Mercer,1111,Trenton,129,State Highway 129,1.0,1.0,0.0,0.0,0.0,NaN,NaN,2024-01-03 12:14:00-05:00
12932,01,Atlantic,0112,Hamilton Twsp,446,Atlantic City Expressway MP 20.5,1.0,1.0,0.0,0.0,0.0,NaN,0.0,2024-01-05 09:14:00-05:00
12933,02,Bergen,0248,Ramsey Boro,NaN,East Oak St,1.0,1.0,0.0,0.0,0.0,East Oak St,NaN,2024-01-05 14:12:00-05:00
13362,07,Essex,0701,Belleville Town,647,County 647 MP .76,1.0,0.0,0.0,1.0,0.0,NaN,NaN,2024-01-05 18:08:00-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13567,08,Gloucester,0818,Washington Twsp,654,County 654,1.0,0.0,0.0,1.0,0.0,NaN,NaN,2024-11-11 20:26:00-05:00
13569,07,Essex,0714,Newark City,508,County 508,1.0,0.0,0.0,1.0,0.0,NaN,NaN,2024-11-12 19:47:00-05:00
13571,20,Union,2001,Berkeley Heights Tw,NaN,Plainfield Ave,1.0,1.0,0.0,0.0,0.0,Plainfield Ave,NaN,2024-11-13 17:45:00-05:00


Load previous year YTD and final counts, and current year YTD counts:

In [5]:
prv_year = ytd.prv_year
cur_year = ytd.cur_year

prv_ytc = to_ytc(prv_crashes)
prv_ytd = prv_ytc.loc[prv_year]

prv_end = to_ytc(ytd.prv_end_crashes).loc[prv_year]

cur_ytc = to_ytc(ytd.cur_ytd_crashes)
cur_ytd = cur_ytc.loc[cur_year]

cur_ytd

,driver,passenger,pedestrian,cyclist,crashes
county,,,,,
Atlantic,17,4,9,0,29
Bergen,21,1,4,0,26
Burlington,22,5,7,1,33
Camden,19,3,12,2,35
Cape May,9,3,0,0,11
Cumberland,9,2,6,0,16
Essex,10,5,24,1,40
Gloucester,18,4,3,1,23
Hudson,7,1,6,2,14


Compute fraction of current year that has elapsed (year-to-date; "YTD") and still remains (rest-of-year; "ROY"):

In [6]:
cur_ytd_frac = ytd.cur_year_frac
cur_roy_frac = 1 - cur_ytd_frac
print('%.1f%% through the year, %.1f%% remaining' % (ytd.cur_year_frac * 100, ytd.cur_roy_frac * 100))

87.8% through the year, 12.2% remaining


Combine previous year YTD / end with current year YTD:

In [7]:
def melt(df, name):
    return (
        df
        .melt(ignore_index=False, var_name='type')
        .set_index('type', append=True)
        .value
        .rename(name)
    )

z = sxs(
    melt(prv_ytd, 'prv_ytd'),
    melt(prv_end, 'prv_end'),
    melt(cur_ytd, 'cur_ytd'),
).fillna(0).astype(int)
z

,,prv_ytd,prv_end,cur_ytd
county,type,,,
Atlantic,driver,27,32,17
Bergen,driver,15,14,21
Burlington,driver,27,31,22
Camden,driver,16,19,19
Cape May,driver,7,8,9
...,...,...,...,...
Salem,crashes,12,14,17
Somerset,crashes,18,18,12
Sussex,crashes,11,12,11


In [8]:
def project_roy(r):
    return int(round(projected_roy_deaths(r.prv_ytd, r.prv_end, r.cur_ytd, cur_ytd_frac)))

roy = z.apply(project_roy, axis=1).rename('roy')
projected = (
    (z.cur_ytd + roy)
    .rename('projected')
    .reset_index(level=1)
    .pivot(columns='type', values='projected')
)
projected

type,crashes,cyclist,driver,passenger,pedestrian
county,,,,,
Atlantic,34,0,20,4,11
Bergen,31,0,20,1,6
Burlington,39,1,25,5,10
Camden,39,2,22,4,12
Cape May,14,0,10,3,0
Cumberland,16,0,9,2,6
Essex,46,1,12,10,27
Gloucester,29,1,20,6,4
Hudson,17,0,8,1,10


In [9]:
projected.to_csv(PROJECTED_CSV)